In [62]:
import pandas as pd
import numpy as np
import re
from connection_keys import connection

In [2]:
connectionbd = connection('db_hardware')

In [39]:
query = """
    select * from graphic_card
"""
df = pd.read_sql(query, connectionbd)

In [42]:
df.dtypes

id                       int64
marca                   object
nome_produto            object
preco_antigo_cartao    float64
preco_atual_cartao     float64
preco_boleto           float64
data                    object
hora                    object
loja                    object
dtype: object

In [35]:
df

,id,marca,nome_produto,preco_antigo_cartao,preco_atual_cartao,preco_boleto,data,hora,loja
0,0,Gigabyte,Placa de Vídeo Gigabyte NVIDIA GeForce GTX 165...,0,"1.399,89","1.329,90",2020-09-06,17:59:00,Kabum
1,1,Asus,Placa de Vídeo Asus TUF NVIDIA GeForce RTX 206...,"2.894,00","2.658,71","2.259,90",2020-09-06,17:59:00,Kabum
2,2,Asus,Placa de Vídeo Asus TUF3 NVIDIA GeForce GTX 16...,0,"2.420,95","2.299,90",2020-09-06,17:59:00,Kabum
3,3,Gigabyte,Placa de Vídeo Gigabyte NVIDIA GeForce GTX 165...,0,"1.578,84","1.499,90",2020-09-06,17:59:00,Kabum
4,4,Gigabyte,Placa de Vídeo Gigabyte GTX 1660 Super OC NVID...,0,"2.210,42","2.099,90",2020-09-06,17:59:00,Kabum
...,...,...,...,...,...,...,...,...,...
372,372,Pny,"Placa de Vídeo PNY NVIDIA Quadro P400, 2GB, GD...",0,"1.311,65","1.114,90",2020-09-06,17:59:00,Kabum
373,373,Powercolor,Placa de Vídeo PowerColor Red Dragon AMD Radeo...,0,"783,41","665,90",2020-09-06,17:59:00,Kabum
374,374,MSI,Placa de Vídeo MSI NVIDIA GeForce GTX 1650 Gam...,"1.599,88","1.388,12","1.179,90",2020-09-06,17:59:00,Kabum
375,375,ASRock,Placa de Vídeo ASRock AMD Radeon RX 5700 XT Ph...,0,"3.835,18","3.259,90",2020-09-06,17:59:00,Kabum


In [40]:
df['preco_antigo_cartao'] = df['preco_antigo_cartao'].str.replace('.', '').str.replace(',', '.').astype(float)
df['preco_atual_cartao'] = df['preco_atual_cartao'].str.replace('.', '').str.replace(',', '.').astype(float)
df['preco_boleto'] = df['preco_boleto'].str.replace('.', '').str.replace(',', '.').astype(float)

In [41]:
df_marcas = df['preco_atual_cartao'].groupby(df['marca'])
print(df_marcas.mean())

marca
ASRock        2285.855385
Afox           403.817500
Amd           5921.450000
Aorus         9212.824000
Asus          3006.544848
Colorful      4827.015000
Deepcool       105.760000
Evga          3473.597455
Gainward      4875.514286
Galax         3408.282000
Gigabyte      2861.453375
Husky          823.410000
Liketec         23.410000
MSI           2576.507368
PCYes          525.448889
Pny           4151.111818
Powercolor    1702.921667
Rise Mode       37.775294
Sapphire      2194.690667
XFX           2115.449231
Zotac         2322.942000
Name: preco_atual_cartao, dtype: float64


In [55]:
df['nome_produto'][df['marca'] == 'Aorus']

217    Placa de Vídeo Gigabyte Aorus NVIDIA GeForce R...
235    Placa de Vídeo Gigabyte Aorus NVIDIA GeForce R...
238    Placa de Vídeo Gigabyte Aorus NVIDIA GeForce R...
321    Placa de Vídeo Gigabyte Aorus NVIDIA GeForce R...
347    Placa de Vídeo Aorus AMD Radeon RX 5700 XT, 8G...
Name: nome_produto, dtype: object

In [140]:
df[df['nome_produto'].str.contains("730")]

,id,marca,nome_produto,preco_antigo_cartao,preco_atual_cartao,preco_boleto,data,hora,loja
17,17,Gigabyte,"Placa de Vídeo Gigabyte NVIDIA GeForce GT 730,...",588.12,470.47,399.9,2020-09-06,17:59:00,Kabum
47,47,Gigabyte,"Placa de Vídeo Gigabyte NVIDIA GeForce GT 730,...",588.12,470.47,399.9,2020-09-06,17:59:00,Kabum
202,202,Zotac,Placa de Vídeo Zotac NVIDIA GeForce GT 730 4GB...,0.00,725.76,616.9,2020-09-06,17:59:00,Kabum
259,259,Evga,"Placa de Vídeo EVGA NVIDIA GeForce GT 730 2GB,...",0.00,352.82,299.9,2020-09-06,17:59:00,Kabum
287,287,MSI,"Placa de Vídeo MSI NVIDIA GeForce GT 730 2GB, ...",0.00,517.53,439.9,2020-09-06,17:59:00,Kabum


In [157]:
def get_resume_product_name(name):
    # string = df['nome_produto'][df['id'] == 259].item()
    pattern = "(RTX|GTX|GT|RX|WX|Quadro|R7|R5|R9|HD)(\d+)"
    resume_name = re.search(pattern, name.replace(" ", ""))
    if resume_name is None: 
        return '' 
    else: 
        return resume_name.group()

df['nome_produto_resumido'] = df.apply(lambda x: get_resume_product_name(x['nome_produto']), axis=1)

In [158]:
df[df['nome_produto_resumido'] == '']
# df

,id,marca,nome_produto,preco_antigo_cartao,preco_atual_cartao,preco_boleto,data,hora,loja,nome_produto_resumido
7,7,Rise Mode,Suporte para Placa de Vídeo VGA Rise até 2 Pla...,31.47,27.26,25.9,2020-09-06,17:59:00,Kabum,
20,20,Rise Mode,Suporte para Placa de Vídeo VGA Rise Transpare...,0.00,39.89,35.9,2020-09-06,17:59:00,Kabum,
28,28,Rise Mode,Suporte para Placa de Vídeo RISE LV2 Small Bla...,54.00,44.59,37.9,2020-09-06,17:59:00,Kabum,
37,37,Rise Mode,Suporte para Placa de Vídeo VGA Rise até 2 Pla...,31.47,27.26,25.9,2020-09-06,17:59:00,Kabum,
50,50,Rise Mode,Suporte para Placa de Vídeo VGA Rise Transpare...,0.00,39.89,35.9,2020-09-06,17:59:00,Kabum,
58,58,Rise Mode,Suporte para Placa de Vídeo RISE LV2 Small Bla...,54.00,44.59,37.9,2020-09-06,17:59:00,Kabum,
66,66,Rise Mode,Suporte para Placa de Video Rise Mode Small Aq...,0.00,46.94,39.9,2020-09-06,17:59:00,Kabum,
72,72,Rise Mode,Suporte Para Placa de Vídeo RISE Snow Edition ...,46.94,29.29,24.9,2020-09-06,17:59:00,Kabum,
74,74,Rise Mode,Suporte para Placa de Video VGA Rise Mode lv2 ...,58.71,42.24,35.9,2020-09-06,17:59:00,Kabum,
80,80,Rise Mode,Suporte para placa de vídeo Rise Mode TecLab E...,0.00,42.24,35.9,2020-09-06,17:59:00,Kabum,
